In [1]:
from tickers import Ticker, scrape_tickers, query_list, recent_IPO_list, upcoming_IPO_list

# Scrape to get updated tickers and company names for stocks and ETFs
scrape_tickers()

import_lists = [query_list, recent_IPO_list, upcoming_IPO_list]

tickers = []

# Build raw query list with ticker abbrevation and name of company
for i in import_lists:
    for item in i:
       # if item.abbrev == 'A' or item.abbrev == 'I':
          #  item.abbrev = '-'
        tickers.append(item)

In [2]:
import pandas as pd

submissions_df = pd.read_csv('submissions.csv')
comments_df = pd.read_csv('comments.csv')

In [3]:
submissions_df['Body_Words'] = submissions_df.Body.str.replace("'", "").str.strip().str.split('[\W_]+')
submissions_df['Title_Words'] = submissions_df.Title.str.replace("'", "").str.strip().str.split('[\W_]+')

In [4]:
common_words = ['Corp', 'Corporation', 'Ltd', 'Acquisition', 'Pharmaceuticals', 'Holding', 'Group']
new_name = ""
ticker_names = []
for ticker in tickers:
    new_name = ""
    company_words = ticker.name.split()
    for word in company_words:
        if word in common_words:
            continue
        else:
            if new_name == "":
                new_name = word
            else:
                new_name = new_name + " " + word
    ticker.name = new_name

In [5]:
submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words
0,Submission,me6ezj,Drinking and Trading don’t mix. I accidentally...,CostantlyLost,Title says it all. Had a few too many drinks o...,NaN,NaN,208,t3_me6ezj,https://www.reddit.com/r/stocks/comments/me6ez...,1298,0.95,[1616818448.0],"[Title, says, it, all, Had, a, few, too, many,...","[Drinking, and, Trading, don, t, mix, I, accid..."
1,Submission,me2d54,"Here is a Market Recap for today Friday, March...",psychotrader00,"\n\n**PsychoMarket Recap - Friday, March 26, ...",NaN,NaN,21,t3_me2d54,https://www.reddit.com/r/stocks/comments/me2d5...,72,0.92,[1616804555.0],"[, PsychoMarket, Recap, Friday, March, 26, 202...","[Here, is, a, Market, Recap, for, today, Frida..."
2,Submission,me8kb3,5 Considerations When Picking Stocks for Long-...,MinnesotaPower,"Like many of you, I started actively investing...",Advice,NaN,18,t3_me8kb3,https://www.reddit.com/r/stocks/comments/me8kb...,81,0.87,[1616827758.0],"[Like, many, of, you, I, started, actively, in...","[5, Considerations, When, Picking, Stocks, for..."
3,Submission,me1wx4,I believe two contradictory things about stock...,NotLegallyBinding,"First, I'm convinced that all the relevant, ac...",NaN,NaN,97,t3_me1wx4,https://www.reddit.com/r/stocks/comments/me1wx...,61,0.82,[1616803173.0],"[First, Im, convinced, that, all, the, relevan...","[I, believe, two, contradictory, things, about..."
4,Submission,med6uw,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Saturday morning to all of you here on r/...,NaN,NaN,19,t3_med6uw,https://www.reddit.com/r/stocks/comments/med6u...,97,0.96,[1616849490.0],"[Good, Saturday, morning, to, all, of, you, he...","[Wall, Street, Week, Ahead, for, the, trading,..."


In [6]:
submissions_df["Tickers"] = ""
for index, row in submissions_df.iterrows():
    tickers_found = []
    body_list = row[12]
    title_list = row[13]
    body_set = set(body_list)
    title_set = set(title_list)
    for ticker in tickers:
        if ticker.abbrev == 'A' or ticker.abbrev == 'I':
            #if (ticker.name.split()[0] in body_set) or (ticker.name.split()[0] in title_set):
            if (ticker.name in body_set) or (ticker.name in title_set):
                tickers_found.append(ticker.abbrev)
        else:
            if (ticker.abbrev in body_set) or (ticker.abbrev in title_set):
                #print("Abbreviation match")
                #print(f"{index}: {ticker.abbrev}")
                if ticker.abbrev not in tickers_found:
                    tickers_found.append(ticker.abbrev)
            elif (ticker.name in body_set) or (ticker.name in title_set):
                #print("Name match")
                #print(f"{index}: {ticker.name}")
                #print(f"{index}: {ticker.abbrev}")
                if ticker.abbrev not in tickers_found:
                    tickers_found.append(ticker.abbrev)
    if len(tickers_found) == 0:
        submissions_df.at[index, "Tickers"] = ""
    else:
        submissions_df.at[index, "Tickers"]= tickers_found
        print(f"{index}: {tickers_found}")

1: ['ADS', 'BCS', 'DRI', 'HON', 'JP', 'MS', 'NDAQ', 'NIO', 'ORLY', 'REGN', 'RH', 'SGFY', 'TLS', 'U', 'DIA', 'QQQ', 'SPY']
2: ['DG', 'E', 'GOOGL', 'HD', 'SPGI', 'TMO', 'TXN', 'UNP', 'V']
3: ['PS']
4: ['AM', 'AMAT', 'BB', 'BNTX', 'CHWY', 'CIO', 'ENPH', 'ET', 'ETSY', 'FCX', 'FOR', 'GILD', 'KMX', 'KNDI', 'L', 'LB', 'LPL', 'LULU', 'MKC', 'MKC.V', 'MTD', 'MU', 'NDAQ', 'NEXT', 'NWG', 'OPEN', 'PM', 'SE', 'TREE', 'TSLA', 'TWOA', 'U', 'VIAC', 'VIACA', 'WBA']
6: ['FANG', 'PLTR', 'PS']
7: ['MX']
10: ['USD']
11: ['DD']
12: ['VIAC']
13: ['CEO', 'FRO', 'NAT', 'STNG', 'TK', 'TNK']
14: ['ARKK']
15: ['F', 'NIO', 'USD']
16: ['DD']
17: ['AMT', 'CCI', 'CTRE', 'EPRT', 'EQIX', 'INVH', 'NSA', 'PLD', 'PSB', 'REXR', 'REIT']
19: ['TSLA', 'VWAGY']
20: ['TWTR']
22: ['GME']
23: ['H', 'IJR', 'IWC', 'IWM', 'SDD']
28: ['CP']
29: ['TD']
30: ['NDAQ', 'TA', 'TWOA']
31: ['X', 'Y']
32: ['TMAC']
34: ['MSFT']
36: ['GME', 'TWOA', 'ARKK', 'LIT', 'MSOS', 'YOLO']
38: ['CEO', 'DHT', 'DSX', 'FRO', 'NAT', 'SFL', 'TNP', 'TWOA']
39: 

In [7]:
submissions_df = submissions_df.loc[submissions_df['Tickers'] != ""]
submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers
1,Submission,me2d54,"Here is a Market Recap for today Friday, March...",psychotrader00,"\n\n**PsychoMarket Recap - Friday, March 26, ...",NaN,NaN,21,t3_me2d54,https://www.reddit.com/r/stocks/comments/me2d5...,72,0.92,[1616804555.0],"[, PsychoMarket, Recap, Friday, March, 26, 202...","[Here, is, a, Market, Recap, for, today, Frida...","[ADS, BCS, DRI, HON, JP, MS, NDAQ, NIO, ORLY, ..."
2,Submission,me8kb3,5 Considerations When Picking Stocks for Long-...,MinnesotaPower,"Like many of you, I started actively investing...",Advice,NaN,18,t3_me8kb3,https://www.reddit.com/r/stocks/comments/me8kb...,81,0.87,[1616827758.0],"[Like, many, of, you, I, started, actively, in...","[5, Considerations, When, Picking, Stocks, for...","[DG, E, GOOGL, HD, SPGI, TMO, TXN, UNP, V]"
3,Submission,me1wx4,I believe two contradictory things about stock...,NotLegallyBinding,"First, I'm convinced that all the relevant, ac...",NaN,NaN,97,t3_me1wx4,https://www.reddit.com/r/stocks/comments/me1wx...,61,0.82,[1616803173.0],"[First, Im, convinced, that, all, the, relevan...","[I, believe, two, contradictory, things, about...",[PS]
4,Submission,med6uw,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Saturday morning to all of you here on r/...,NaN,NaN,19,t3_med6uw,https://www.reddit.com/r/stocks/comments/med6u...,97,0.96,[1616849490.0],"[Good, Saturday, morning, to, all, of, you, he...","[Wall, Street, Week, Ahead, for, the, trading,...","[AM, AMAT, BB, BNTX, CHWY, CIO, ENPH, ET, ETSY..."
6,Submission,mdwibp,PLTR - bearish discussion to better understand...,wsbloverrrrrr,"(Please excuse my username, it's the only acco...",Company Discussion,NaN,58,t3_mdwibp,https://www.reddit.com/r/stocks/comments/mdwib...,43,0.79,[1616787276.0],"[, Please, excuse, my, username, its, the, onl...","[PLTR, bearish, discussion, to, better, unders...","[FANG, PLTR, PS]"


In [8]:
import nltk
#nltk.download('punkt')

In [9]:
# Using Vader Sentiment Analysis on Submissions
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from pprint import pprint
#nltk.download('vader_lexicon')
sia = SIA()
results = []
submissions_df['negative_score'] = ""
submissions_df['neutral_score'] = ""
submissions_df['positive_score'] = ""
submissions_df['compound_score'] = ""
#submissions_loop = submissions_df[['Title', 'Body', 'Body_Words', 'Title_Words', 'Tickers']]
for index, row in submissions_df.iterrows():
    title = row[2]
    body = row[4]
    eval_text = title + " " + body
    body_words = row[12]
    title_words = row[13]
    tickers = row[14]
    pol_score= sia.polarity_scores(eval_text)
    #pol_score['submission_text'] = eval_text
    #pol_score['tickers'] = tickers
    #results.append(pol_score)
    submissions_df.at[index, 'negative_score'] = pol_score['neg']
    submissions_df.at[index, 'neutral_score'] = pol_score['neu']
    submissions_df.at[index, 'positive_score'] = pol_score['pos']
    submissions_df.at[index, 'compound_score'] = pol_score['compound']
        #submissions_loop['sentiment_scores'] = pol_score
#pprint(results[:3], width=100)

submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score
1,Submission,me2d54,"Here is a Market Recap for today Friday, March...",psychotrader00,"\n\n**PsychoMarket Recap - Friday, March 26, ...",NaN,NaN,21,t3_me2d54,https://www.reddit.com/r/stocks/comments/me2d5...,72,0.92,[1616804555.0],"[, PsychoMarket, Recap, Friday, March, 26, 202...","[Here, is, a, Market, Recap, for, today, Frida...","[ADS, BCS, DRI, HON, JP, MS, NDAQ, NIO, ORLY, ...",0.045,0.848,0.107,0.9937
2,Submission,me8kb3,5 Considerations When Picking Stocks for Long-...,MinnesotaPower,"Like many of you, I started actively investing...",Advice,NaN,18,t3_me8kb3,https://www.reddit.com/r/stocks/comments/me8kb...,81,0.87,[1616827758.0],"[Like, many, of, you, I, started, actively, in...","[5, Considerations, When, Picking, Stocks, for...","[DG, E, GOOGL, HD, SPGI, TMO, TXN, UNP, V]",0.042,0.836,0.122,0.9842
3,Submission,me1wx4,I believe two contradictory things about stock...,NotLegallyBinding,"First, I'm convinced that all the relevant, ac...",NaN,NaN,97,t3_me1wx4,https://www.reddit.com/r/stocks/comments/me1wx...,61,0.82,[1616803173.0],"[First, Im, convinced, that, all, the, relevan...","[I, believe, two, contradictory, things, about...",[PS],0.142,0.778,0.081,-0.7559
4,Submission,med6uw,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Saturday morning to all of you here on r/...,NaN,NaN,19,t3_med6uw,https://www.reddit.com/r/stocks/comments/med6u...,97,0.96,[1616849490.0],"[Good, Saturday, morning, to, all, of, you, he...","[Wall, Street, Week, Ahead, for, the, trading,...","[AM, AMAT, BB, BNTX, CHWY, CIO, ENPH, ET, ETSY...",0.031,0.871,0.098,0.9999
6,Submission,mdwibp,PLTR - bearish discussion to better understand...,wsbloverrrrrr,"(Please excuse my username, it's the only acco...",Company Discussion,NaN,58,t3_mdwibp,https://www.reddit.com/r/stocks/comments/mdwib...,43,0.79,[1616787276.0],"[, Please, excuse, my, username, its, the, onl...","[PLTR, bearish, discussion, to, better, unders...","[FANG, PLTR, PS]",0.066,0.746,0.188,0.9981


In [10]:
submissions_df['sentiment_label'] = 0
submissions_df.loc[submissions_df['compound_score'] > 0.2, 'sentiment_label'] = 1
submissions_df.loc[submissions_df['compound_score'] < -0.2, 'sentiment_label'] = -1
submissions_df

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,...,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
1,Submission,me2d54,"Here is a Market Recap for today Friday, March...",psychotrader00,"\n\n**PsychoMarket Recap - Friday, March 26, ...",NaN,NaN,21,t3_me2d54,https://www.reddit.com/r/stocks/comments/me2d5...,...,0.92,[1616804555.0],"[, PsychoMarket, Recap, Friday, March, 26, 202...","[Here, is, a, Market, Recap, for, today, Frida...","[ADS, BCS, DRI, HON, JP, MS, NDAQ, NIO, ORLY, ...",0.045,0.848,0.107,0.9937,1
2,Submission,me8kb3,5 Considerations When Picking Stocks for Long-...,MinnesotaPower,"Like many of you, I started actively investing...",Advice,NaN,18,t3_me8kb3,https://www.reddit.com/r/stocks/comments/me8kb...,...,0.87,[1616827758.0],"[Like, many, of, you, I, started, actively, in...","[5, Considerations, When, Picking, Stocks, for...","[DG, E, GOOGL, HD, SPGI, TMO, TXN, UNP, V]",0.042,0.836,0.122,0.9842,1
3,Submission,me1wx4,I believe two contradictory things about stock...,NotLegallyBinding,"First, I'm convinced that all the relevant, ac...",NaN,NaN,97,t3_me1wx4,https://www.reddit.com/r/stocks/comments/me1wx...,...,0.82,[1616803173.0],"[First, Im, convinced, that, all, the, relevan...","[I, believe, two, contradictory, things, about...",[PS],0.142,0.778,0.081,-0.7559,-1
4,Submission,med6uw,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Saturday morning to all of you here on r/...,NaN,NaN,19,t3_med6uw,https://www.reddit.com/r/stocks/comments/med6u...,...,0.96,[1616849490.0],"[Good, Saturday, morning, to, all, of, you, he...","[Wall, Street, Week, Ahead, for, the, trading,...","[AM, AMAT, BB, BNTX, CHWY, CIO, ENPH, ET, ETSY...",0.031,0.871,0.098,0.9999,1
6,Submission,mdwibp,PLTR - bearish discussion to better understand...,wsbloverrrrrr,"(Please excuse my username, it's the only acco...",Company Discussion,NaN,58,t3_mdwibp,https://www.reddit.com/r/stocks/comments/mdwib...,...,0.79,[1616787276.0],"[, Please, excuse, my, username, its, the, onl...","[PLTR, bearish, discussion, to, better, unders...","[FANG, PLTR, PS]",0.066,0.746,0.188,0.9981,1
7,Submission,mdvvkm,Opportunity to gain about 10-14% on $MX (Bough...,RowanHarley,"Right now, $MX is trading far below what it wa...",Company Discussion,NaN,52,t3_mdvvkm,https://www.reddit.com/r/stocks/comments/mdvvk...,...,0.85,[1616785531.0],"[Right, now, MX, is, trading, far, below, what...","[Opportunity, to, gain, about, 10, 14, on, MX,...",[MX],0.085,0.763,0.152,0.8927,1
10,Submission,me0ast,Some figures summarizing what has been happeni...,futureIsYes,The Hang Seng Index contains all the Chinese g...,NaN,NaN,62,t3_me0ast,https://www.reddit.com/r/stocks/comments/me0as...,...,0.81,[1616798179.0],"[The, Hang, Seng, Index, contains, all, the, C...","[Some, figures, summarizing, what, has, been, ...",[USD],0.11,0.89,0,-0.8963,-1
11,Submission,mebzea,What stocks focus on sexual self-satisfaction?,Smur_,"To explain myself, I really do feel like this ...",Industry Discussion,NaN,74,t3_mebzea,https://www.reddit.com/r/stocks/comments/mebze...,...,0.72,[1616844550.0],"[To, explain, myself, I, really, do, feel, lik...","[What, stocks, focus, on, sexual, self, satisf...",[DD],0.03,0.843,0.127,0.9194,1
12,Submission,mdtss5,Anyone own Viacom (VIAC) here?,rockinoutwith2,"If so, my condolences...damn, this stock just ...",NaN,NaN,56,t3_mdtss5,https://www.reddit.com/r/stocks/comments/mdtss...,...,0.90,[1616779789.0],"[If, so, my, condolences, damn, this, stock, j...","[Anyone, own, Viacom, VIAC, here, ]",[VIAC],0.05,0.785,0.165,0.6129,1
13,Submission,meaofc,Suez Canal Tanker Play,TimThyTurtle,$STNG $TNK $NAT $TK $FRO\nThe Suez Canal block...,Industry Discussion,NaN,32,t3_meaofc,https://www.reddit.com/r/stocks/comments/meaof...,...,0.74,[1616838462.0],"[, STNG, TNK, NAT, TK, FRO, The, Suez, Canal, ...","[Suez, Canal, Tanker, Play]","[CEO, FRO, NAT,

In [22]:
submissions_df['sentiment_weight'] = submissions_df['Score'] * submissions_df['Upvote_Ratio']

In [23]:
submissions_df

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,...,Created_Date_UTC,Body_Words,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label,sentiment_weight
1,Submission,me2d54,"Here is a Market Recap for today Friday, March...",psychotrader00,"\n\n**PsychoMarket Recap - Friday, March 26, ...",NaN,NaN,21,t3_me2d54,https://www.reddit.com/r/stocks/comments/me2d5...,...,[1616804555.0],"[, PsychoMarket, Recap, Friday, March, 26, 202...","[Here, is, a, Market, Recap, for, today, Frida...","[ADS, BCS, DRI, HON, JP, MS, NDAQ, NIO, ORLY, ...",0.045,0.848,0.107,0.9937,1,66.24
2,Submission,me8kb3,5 Considerations When Picking Stocks for Long-...,MinnesotaPower,"Like many of you, I started actively investing...",Advice,NaN,18,t3_me8kb3,https://www.reddit.com/r/stocks/comments/me8kb...,...,[1616827758.0],"[Like, many, of, you, I, started, actively, in...","[5, Considerations, When, Picking, Stocks, for...","[DG, E, GOOGL, HD, SPGI, TMO, TXN, UNP, V]",0.042,0.836,0.122,0.9842,1,70.47
3,Submission,me1wx4,I believe two contradictory things about stock...,NotLegallyBinding,"First, I'm convinced that all the relevant, ac...",NaN,NaN,97,t3_me1wx4,https://www.reddit.com/r/stocks/comments/me1wx...,...,[1616803173.0],"[First, Im, convinced, that, all, the, relevan...","[I, believe, two, contradictory, things, about...",[PS],0.142,0.778,0.081,-0.7559,-1,50.02
4,Submission,med6uw,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Saturday morning to all of you here on r/...,NaN,NaN,19,t3_med6uw,https://www.reddit.com/r/stocks/comments/med6u...,...,[1616849490.0],"[Good, Saturday, morning, to, all, of, you, he...","[Wall, Street, Week, Ahead, for, the, trading,...","[AM, AMAT, BB, BNTX, CHWY, CIO, ENPH, ET, ETSY...",0.031,0.871,0.098,0.9999,1,93.12
6,Submission,mdwibp,PLTR - bearish discussion to better understand...,wsbloverrrrrr,"(Please excuse my username, it's the only acco...",Company Discussion,NaN,58,t3_mdwibp,https://www.reddit.com/r/stocks/comments/mdwib...,...,[1616787276.0],"[, Please, excuse, my, username, its, the, onl...","[PLTR, bearish, discussion, to, better, unders...","[FANG, PLTR, PS]",0.066,0.746,0.188,0.9981,1,33.97
7,Submission,mdvvkm,Opportunity to gain about 10-14% on $MX (Bough...,RowanHarley,"Right now, $MX is trading far below what it wa...",Company Discussion,NaN,52,t3_mdvvkm,https://www.reddit.com/r/stocks/comments/mdvvk...,...,[1616785531.0],"[Right, now, MX, is, trading, far, below, what...","[Opportunity, to, gain, about, 10, 14, on, MX,...",[MX],0.085,0.763,0.152,0.8927,1,34.85
10,Submission,me0ast,Some figures summarizing what has been happeni...,futureIsYes,The Hang Seng Index contains all the Chinese g...,NaN,NaN,62,t3_me0ast,https://www.reddit.com/r/stocks/comments/me0as...,...,[1616798179.0],"[The, Hang, Seng, Index, contains, all, the, C...","[Some, figures, summarizing, what, has, been, ...",[USD],0.11,0.89,0,-0.8963,-1,28.35
11,Submission,mebzea,What stocks focus on sexual self-satisfaction?,Smur_,"To explain myself, I really do feel like this ...",Industry Discussion,NaN,74,t3_mebzea,https://www.reddit.com/r/stocks/comments/mebze...,...,[1616844550.0],"[To, explain, myself, I, really, do, feel, lik...","[What, stocks, focus, on, sexual, self, satisf...",[DD],0.03,0.843,0.127,0.9194,1,23.76
12,Submission,mdtss5,Anyone own Viacom (VIAC) here?,rockinoutwith2,"If so, my condolences...damn, this stock just ...",NaN,NaN,56,t3_mdtss5,https://www.reddit.com/r/stocks/comments/mdtss...,...,[1616779789.0],"[If, so, my, condolences, damn, this, stock, j...","[Anyone, own, Viacom, VIAC, here, ]",[VIAC],0.05,0.785,0.165,0.6129,1,24.30
13,Submission,meaofc,Suez Canal Tanker Play,TimThyTurtle,$STNG $TNK $NAT $TK $FRO\nThe Suez Canal block...,Industry Discussion,NaN,32,t3_meaofc,https://www.reddit.com/r/stocks/comments/meaof...,...,[1616838462.0],"[, STNG, TNK, NAT, TK, FRO, The, Suez, Canal, ...","[Suez, Canal, Tanker, Play]","[CEO, F

In [24]:
comments_df.head()

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Flair,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
187,Comment,me2d54,gsd1r54,t3_me2d54,pman6,['ok guys....\n\nwhat the fuck happened during...,NaN,NaN,False,35,[1616805654.0],"[, ok, guys, n, nwhat, the, fuck, happened, du...","[Here, is, a, Market, Recap, for, today, Frida...",0.224,0.776,0,-0.7236,-1
188,Comment,me2d54,gsdbbya,t3_me2d54,NeelAsman,"['Shout out for yesterday’s post, you nailed S...",NaN,NaN,False,5,[1616809931.0],"[, Shout, out, for, yesterday, s, post, you, n...","[Here, is, a, Market, Recap, for, today, Frida...",0,1,0,0,0
189,Comment,me2d54,gsd2vy0,t3_me2d54,thatsjetfuel,"[""I'm not seeing how TSM is a bad play at this...",NaN,NaN,False,7,[1616806145.0],"[, Im, not, seeing, how, TSM, is, a, bad, play...","[Here, is, a, Market, Recap, for, today, Frida...",0.147,0.767,0.087,-0.5423,-1
190,Comment,me2d54,gsd61xo,t1_gsd1r54,hitmon_gg,['God wanted me to have a better weekend?'],NaN,NaN,False,23,[1616807556.0],"[, God, wanted, me, to, have, a, better, weeke...","[Here, is, a, Market, Recap, for, today, Frida...",0,0.674,0.326,0.4404,1
191,Comment,me2d54,gsddp75,t1_gsd1r54,bothodler,"[""I don't want to get super conspiracy-ie but ...",NaN,NaN,False,10,[1616811018.0],"[, I, dont, want, to, get, super, conspiracy, ...","[Here, is, a, Market, Recap, for, today, Frida...",0.299,0.598,0.102,-0.9497,-1


In [25]:
comments_df['Body_Words'] = comments_df.Body.str.replace("'", "").str.strip().str.split('[\W_]+')

In [26]:
comments_df['Tickers'] = ""
for index_c, row_c in comments_df.iterrows():
    for index_s, row_s in submissions_df.iterrows():
        if row_c[1] == row_s[1]:
             comments_df.at[index_c, 'Tickers'] = row_s[14]

In [27]:
comments_df = comments_df.loc[comments_df["Tickers"] != ""]
comments_df.head()

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Flair,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
187,Comment,me2d54,gsd1r54,t3_me2d54,pman6,['ok guys....\n\nwhat the fuck happened during...,NaN,NaN,False,35,[1616805654.0],"[, ok, guys, n, nwhat, the, fuck, happened, du...","[Here, is, a, Market, Recap, for, today, Frida...",0.224,0.776,0,-0.7236,-1
188,Comment,me2d54,gsdbbya,t3_me2d54,NeelAsman,"['Shout out for yesterday’s post, you nailed S...",NaN,NaN,False,5,[1616809931.0],"[, Shout, out, for, yesterday, s, post, you, n...","[Here, is, a, Market, Recap, for, today, Frida...",0,1,0,0,0
189,Comment,me2d54,gsd2vy0,t3_me2d54,thatsjetfuel,"[""I'm not seeing how TSM is a bad play at this...",NaN,NaN,False,7,[1616806145.0],"[, Im, not, seeing, how, TSM, is, a, bad, play...","[Here, is, a, Market, Recap, for, today, Frida...",0.147,0.767,0.087,-0.5423,-1
190,Comment,me2d54,gsd61xo,t1_gsd1r54,hitmon_gg,['God wanted me to have a better weekend?'],NaN,NaN,False,23,[1616807556.0],"[, God, wanted, me, to, have, a, better, weeke...","[Here, is, a, Market, Recap, for, today, Frida...",0,0.674,0.326,0.4404,1
191,Comment,me2d54,gsddp75,t1_gsd1r54,bothodler,"[""I don't want to get super conspiracy-ie but ...",NaN,NaN,False,10,[1616811018.0],"[, I, dont, want, to, get, super, conspiracy, ...","[Here, is, a, Market, Recap, for, today, Frida...",0.299,0.598,0.102,-0.9497,-1


In [28]:
comments_df['negative_score'] = ""
comments_df['neutral_score'] = ""
comments_df['positive_score'] = ""
comments_df['compound_score'] = ""
#submissions_loop = submissions_df[['Title', 'Body', 'Body_Words', 'Title_Words', 'Tickers']]
for index, row in comments_df.iterrows():
    body = row[5]
    tickers = row[11]
    pol_score= sia.polarity_scores(body)
    #pol_score['submission_text'] = eval_text
    #pol_score['tickers'] = tickers
    #results.append(pol_score)
    comments_df.at[index, 'negative_score'] = pol_score['neg']
    comments_df.at[index, 'neutral_score'] = pol_score['neu']
    comments_df.at[index, 'positive_score'] = pol_score['pos']
    comments_df.at[index, 'compound_score'] = pol_score['compound']
        #submissions_loop['sentiment_scores'] = pol_score
#pprint(results[:3], width=100)

comments_df.head()

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Flair,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
187,Comment,me2d54,gsd1r54,t3_me2d54,pman6,['ok guys....\n\nwhat the fuck happened during...,NaN,NaN,False,35,[1616805654.0],"[, ok, guys, n, nwhat, the, fuck, happened, du...","[Here, is, a, Market, Recap, for, today, Frida...",0.224,0.776,0,-0.7236,-1
188,Comment,me2d54,gsdbbya,t3_me2d54,NeelAsman,"['Shout out for yesterday’s post, you nailed S...",NaN,NaN,False,5,[1616809931.0],"[, Shout, out, for, yesterday, s, post, you, n...","[Here, is, a, Market, Recap, for, today, Frida...",0,1,0,0,0
189,Comment,me2d54,gsd2vy0,t3_me2d54,thatsjetfuel,"[""I'm not seeing how TSM is a bad play at this...",NaN,NaN,False,7,[1616806145.0],"[, Im, not, seeing, how, TSM, is, a, bad, play...","[Here, is, a, Market, Recap, for, today, Frida...",0.147,0.767,0.087,-0.5423,-1
190,Comment,me2d54,gsd61xo,t1_gsd1r54,hitmon_gg,['God wanted me to have a better weekend?'],NaN,NaN,False,23,[1616807556.0],"[, God, wanted, me, to, have, a, better, weeke...","[Here, is, a, Market, Recap, for, today, Frida...",0,0.674,0.326,0.4404,1
191,Comment,me2d54,gsddp75,t1_gsd1r54,bothodler,"[""I don't want to get super conspiracy-ie but ...",NaN,NaN,False,10,[1616811018.0],"[, I, dont, want, to, get, super, conspiracy, ...","[Here, is, a, Market, Recap, for, today, Frida...",0.299,0.598,0.102,-0.9497,-1


In [16]:
comments_df['sentiment_label'] = 0
comments_df.loc[comments_df['compound_score'] > 0.2, 'sentiment_label'] = 1
comments_df.loc[comments_df['compound_score'] < -0.2, 'sentiment_label'] = -1
comments_df

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Flair,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
187,Comment,me2d54,gsd1r54,t3_me2d54,pman6,['ok guys....\n\nwhat the fuck happened during...,NaN,NaN,False,35,[1616805654.0],"[, ok, guys, n, nwhat, the, fuck, happened, du...","[Here, is, a, Market, Recap, for, today, Frida...",0.224,0.776,0,-0.7236,-1
188,Comment,me2d54,gsdbbya,t3_me2d54,NeelAsman,"['Shout out for yesterday’s post, you nailed S...",NaN,NaN,False,5,[1616809931.0],"[, Shout, out, for, yesterday, s, post, you, n...","[Here, is, a, Market, Recap, for, today, Frida...",0,1,0,0,0
189,Comment,me2d54,gsd2vy0,t3_me2d54,thatsjetfuel,"[""I'm not seeing how TSM is a bad play at this...",NaN,NaN,False,7,[1616806145.0],"[, Im, not, seeing, how, TSM, is, a, bad, play...","[Here, is, a, Market, Recap, for, today, Frida...",0.147,0.767,0.087,-0.5423,-1
190,Comment,me2d54,gsd61xo,t1_gsd1r54,hitmon_gg,['God wanted me to have a better weekend?'],NaN,NaN,False,23,[1616807556.0],"[, God, wanted, me, to, have, a, better, weeke...","[Here, is, a, Market, Recap, for, today, Frida...",0,0.674,0.326,0.4404,1
191,Comment,me2d54,gsddp75,t1_gsd1r54,bothodler,"[""I don't want to get super conspiracy-ie but ...",NaN,NaN,False,10,[1616811018.0],"[, I, dont, want, to, get, super, conspiracy, ...","[Here, is, a, Market, Recap, for, today, Frida...",0.299,0.598,0.102,-0.9497,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111,Comment,me11cn,gsdhaar,t1_gsdf2zh,merlinsbeers,"[""He isn't self-made. He comes from a wealthy ...",NaN,NaN,False,2,[1616812668.0],"[, He, isnt, self, made, He, comes, from, a, w...","[Elons, pumps, lost, its, effect, Totally, uns...",0,0.846,0.154,0.6249,1
2112,Comment,me11cn,gsge5va,t1_gsdf2zh,harrison_wintergreen,['> That dude is one of the few self made bill...,NaN,NaN,False,1,[1616856792.0],"[, That, dude, is, one, of, the, few, self, ma...","[Elons, pumps, lost, its, effect, Totally, uns...",0,1,0,0,0
2113,Comment,me11cn,gsd6jo3,t1_gscuc3i,Ironleg01,"['Oh wow, news outlet reporting on thing that ...",NaN,NaN,False,4,[1616807785.0],"[, Oh, wow, news, outlet, reporting, on, thing...","[Elons, pumps, lost, its, effect, Totally, uns...",0.296,0.51,0.194,-0.25,-1
2114,Comment,me11cn,gsd0mbr,t1_gsczs3d,Tacoman404,"['Huh, Elon Musk is actually a vole?']",NaN,NaN,False,2,[1616805158.0],"[, Huh, Elon, Musk, is, actually, a, vole, ]","[Elons, pumps, lost, its, effect, Totally, uns...",0,1,0,0,0


In [34]:
sentiment_average_df = comments_df[['Submission_ID','sentiment_label']]

In [36]:
sentiment_average_df['sentiment_weight'] = comments_df['Score']
sentiment_average_df

/Users/madisonleopold/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Submission_ID,sentiment_label,sentiment_weight
187,me2d54,-1,35
188,me2d54,0,5
189,me2d54,-1,7
190,me2d54,1,23
191,me2d54,-1,10
...,...,...,...
2111,me11cn,1,2
2112,me11cn,0,1
2113,me11cn,-1,4
2114,me11cn,0,2


In [37]:
sentiment_average_df['sentiment_adjusted'] = sentiment_average_df['sentiment_label'] * sentiment_average_df['sentiment_weight']

/Users/madisonleopold/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
sentiment_average = sentiment_average_df[['Submission_ID','sentiment_adjusted']].groupby('Submission_ID').mean()

In [41]:
sentiment_average['sentiment_output'] = 0
sentiment_average.loc[sentiment_average['sentiment_adjusted'] > 0.2, 'sentiment_output'] = 1
sentiment_average.loc[sentiment_average['sentiment_adjusted'] < -0.2, 'sentiment_output'] = -1
sentiment_average.reset_index(inplace=True)
#sentiment_average = sentiment_average.set_index('Submission_ID')

In [42]:
sentiment_average

,Submission_ID,sentiment_adjusted,sentiment_output
0,mdqmi5,-2.333333,-1
1,mdr73u,0.461538,1
2,mdrafp,-0.200000,0
3,mdrf39,-0.200000,0
4,mdrs63,1.571429,1
5,mds0i6,0.500000,1
6,mds4bo,0.400000,1
7,mdsfjo,0.888889,1
8,mdssz4,0.083333,0
9,mdt1eq,1.068966,1


In [66]:
#combined_df = submissions_df.join(sentiment_average)
combined_df = pd.merge(submissions_df, sentiment_average, how='left', on='Submission_ID',
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [67]:
combined_df

,Post_Type,Submission_ID,Title,Author,Body,Flair,Distinguished,Num_Comments,Post_ID,URL,...,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label,sentiment_weight,sentiment_adjusted,sentiment_output
0,Submission,mdqmi5,$GSX - Options activity,SaveAmerica21,$GSX - question. What is up with all the Chine...,NaN,NaN,3,t3_mdqmi5,https://www.reddit.com/r/stocks/comments/mdqmi...,...,"[, GSX, Options, activity]","[AMBO, CLEU, EEIQ, FEDU, WAFU]",0,0.957,0.043,0.34,1,3.00,-2.333333,-1.0
1,Submission,mdr73u,Short term tanker stocks for Suez Canal blocka...,Rock_it_Scientist,"Ok, so we all know that some quarter mile lon...",Industry Discussion,NaN,14,t3_mdr73u,https://www.reddit.com/r/stocks/comments/mdr73...,...,"[Short, term, tanker, stocks, for, Suez, Canal...","[CEO, DHT, DSX, FRO, NAT, SFL, TNP, TWOA]",0.08,0.869,0.051,-0.9834,-1,5.39,0.461538,1.0
2,Submission,mdrafp,XPO Logistics Inc. (XPO) Soars 2.43% on March 25,thinkB4WeSpeak,XPO Logistics Inc. (XPO) had a good day on the...,Company News,NaN,5,t3_mdrafp,https://www.reddit.com/r/stocks/comments/mdraf...,...,"[XPO, Logistics, Inc, XPO, Soars, 2, 43, on, M...","[CEO, TWOA, XPO, LTL]",0.026,0.908,0.066,0.9169,1,0.00,-0.200000,0.0
3,Submission,mdrf39,"Rookie Mistake, left with a question.",Cindermidnight2,So I misunderstood RobinHood’s day trading war...,Advice,NaN,57,t3_mdrf39,https://www.reddit.com/r/stocks/comments/mdrf3...,...,"[Rookie, Mistake, left, with, a, question, ]",[TD],0.081,0.769,0.15,0.9669,1,8.47,-0.200000,0.0
4,Submission,mdrs63,Dollar cost average,noeljvt,Is there a way to calculate dollar cost averag...,NaN,NaN,14,t3_mdrs63,https://www.reddit.com/r/stocks/comments/mdrs6...,...,"[Dollar, cost, average]",[TWOA],0,0.905,0.095,0.6046,1,0.00,1.571429,1.0
5,Submission,mds0i6,UVXY Tax situation,mkays10,"Idk If any of you are tax pros, but I’m sure p...",NaN,NaN,2,t3_mds0i6,https://www.reddit.com/r/stocks/comments/mds0i...,...,"[UVXY, Tax, situation]","[K, L, UVXY]",0.008,0.788,0.203,0.9828,1,1.50,0.500000,1.0
6,Submission,mds4bo,Anyone Else Getting Bearish on Midcaps?,JagwarRocker,"Still very early, but seeing a clear left shou...",Trades,NaN,25,t3_mds4bo,https://www.reddit.com/r/stocks/comments/mds4b...,...,"[Anyone, Else, Getting, Bearish, on, Midcaps, ]","[H, IJR, IWC, IWM, SDD]",0.059,0.901,0.04,0.1381,0,7.60,0.400000,1.0
7,Submission,mdsfjo,Can someone point out to me what i'm missing h...,Manyworldsz,Let me know if this is not the place to ask an...,NaN,NaN,9,t3_mdsfjo,https://www.reddit.com/r/stocks/comments/mdsfj...,...,"[Can, someone, point, out, to, me, what, im, m...","[AONE, MT]",0.071,0.775,0.154,0.9845,1,2.13,0.888889,1.0
8,Submission,mdssz4,Trades betting on US/China conflict?,lolkkthxbye,Tension between the US and China over Taiwan s...,Trades,NaN,12,t3_mdssz4,https://www.reddit.com/r/stocks/comments/mdssz...,...,"[Trades, betting, on, US, China, conflict, ]",[RTX],0.13,0.821,0.049,-0.8532,-1,3.90,0.083333,0.0
9,Submission,mdt1eq,[Nio vs F] Why is NIO more hyped than F?,Many_Artichoke9463,NIO 2020 revenue: ~~$16M~~ $2.4B\n\nF 2020 rev...,Company Discussion,NaN,58,t3_mdt1eq,https://www.reddit.com/r/stocks/comments/mdt1e...,...,"[, Nio, vs, F, Why, is, NIO, more, hyped, than...","[F, NIO, USD]",0.068,0.837,0.096,0.6704,1,19.50,1.068966,1.0


In [68]:
combined_df.rename(columns = {'sentiment_adjusted':'comment_sentiment'}, inplace = True)

In [69]:
combined_df['submission_sentiment'] = combined_df['sentiment_label'] * combined_df['sentiment_weight']

In [70]:
combined_df['combined_sentiment'] = combined_df['comment_sentiment'] + combined_df['submission_sentiment']

In [71]:
combined_df['final_sentiment_label'] = 0
combined_df.loc[combined_df['combined_sentiment'] > 0.2, 'final_sentiment_label'] = 1
combined_df.loc[combined_df['combined_sentiment'] < -0.2, 'final_sentiment_label'] = -1
#combined_df.reset_index(inplace=True)

In [73]:
combined_df['Tickers']

0                        [AMBO, CLEU, EEIQ, FEDU, WAFU]
1             [CEO, DHT, DSX, FRO, NAT, SFL, TNP, TWOA]
2                                 [CEO, TWOA, XPO, LTL]
3                                                  [TD]
4                                                [TWOA]
5                                          [K, L, UVXY]
6                               [H, IJR, IWC, IWM, SDD]
7                                            [AONE, MT]
8                                                 [RTX]
9                                         [F, NIO, USD]
10                                               [VIAC]
11                   [GME, TWOA, ARKK, LIT, MSOS, YOLO]
12                                               [ROKU]
13                                                [GME]
14                                           [GME, USD]
15                                                 [MX]
16                                            [TD, WFC]
17                                     [FANG, PL

In [ ]:
# Get running count of tickers mentioned (# of times mentioned) and with average sentiment